# Pré-processamento dos Dados - 20 Newsgroups

**Objetivo:** Carregar as 6 classes da base 20 Newsgroups, aplicar pré-processamento leve e salvar em `/data/processed`.


In [2]:
import pandas as pd
import numpy as np
import pickle
import re
import os
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS


## 1. Carregar as 6 classes selecionadas


In [3]:
# Classes selecionadas
categories = [
    'rec.sport.baseball',
    'rec.sport.hockey',
    'talk.politics.mideast',
    'talk.politics.guns',
    'rec.autos',
    'sci.space'
]

# Carregar dados
newsgroups = fetch_20newsgroups(
    subset='all',
    categories=categories,
    remove=('headers', 'footers', 'quotes'),
    shuffle=True,
    random_state=42
)

print(f"Total de documentos: {len(newsgroups.data)}")
print(f"Classes: {newsgroups.target_names}")
print(f"Distribuição de classes:\n{pd.Series(newsgroups.target).value_counts().sort_index()}")


Total de documentos: 5820
Classes: ['rec.autos', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.space', 'talk.politics.guns', 'talk.politics.mideast']
Distribuição de classes:
0    990
1    994
2    999
3    987
4    910
5    940
Name: count, dtype: int64


## 2. Converter para DataFrame


In [8]:
# Criar DataFrame
df = pd.DataFrame({
    'text': newsgroups.data,
    'target': newsgroups.target,
    'target_name': [newsgroups.target_names[t] for t in newsgroups.target]
})

print(f"Shape do DataFrame: {df.shape}")
print(f"\nPrimeiras linhas:")
df.head()


Shape do DataFrame: (5820, 3)

Primeiras linhas:


,text,target,target_name
0,From: thwang@mentor.cc.purdue.edu (Tommy Hwang...,0,rec.autos
1,"\nOh no! Say it isn't so!\n\n\nYes, Yl|nen is...",2,rec.sport.hockey
2,\nGrant Fuhr rules! -).,2,rec.sport.hockey
3,(Eric Roush) writes...\n\n\n\n\nFunny. I...,1,rec.sport.baseball
4,THE FOLLOWING POSTING WAS FORGED IN MY NAME! P...,5,talk.politics.mideast


## 3. Função de pré-processamento


In [9]:
def preprocess_text(text):
    """
    Aplica pré-processamento leve:
    - Conversão para minúsculas
    - Remoção de pontuação e dígitos
    - Remoção de stopwords
    """
    # Converter para minúsculas
    text = text.lower()
    
    # Remover pontuação e dígitos (manter apenas letras e espaços)
    text = re.sub(r'[^a-z\s]', ' ', text)
    
    # Remover múltiplos espaços
    text = re.sub(r'\s+', ' ', text)
    
    # Dividir em palavras e remover stopwords
    words = text.split()
    words = [w for w in words if w not in ENGLISH_STOP_WORDS and len(w) > 2]
    
    # Juntar novamente
    text = ' '.join(words)
    
    return text.strip()


## 4. Aplicar pré-processamento


In [10]:
# Aplicar pré-processamento (pode demorar alguns segundos)
print("Aplicando pré-processamento...")
df['text_processed'] = df['text'].apply(preprocess_text)

# Remover documentos vazios após pré-processamento
df = df[df['text_processed'].str.len() > 0].reset_index(drop=True)

print(f"Documentos após pré-processamento: {len(df)}")
print(f"\nExemplo de texto original vs processado:")
print(f"\nOriginal:\n{df['text'].iloc[0][:200]}...")
print(f"\nProcessado:\n{df['text_processed'].iloc[0][:200]}...")


Aplicando pré-processamento...
Documentos após pré-processamento: 5611

Exemplo de texto original vs processado:

Original:
From: thwang@mentor.cc.purdue.edu (Tommy Hwang)
Subject: Advise needed in buying Automobile

   >I am in search of a dependable automobile to purchase.  Below
   >are its requirements:
   >	5. V6 or a...

Processado:
thwang mentor purdue edu tommy hwang subject advise needed buying automobile search dependable automobile purchase requirements cars mentioned smaller engine tony...


## 5. Salvar dataset pré-processado


In [11]:
import os
import pickle
import pandas as pd

# Criar diretório se não existir
os.makedirs('../data/processed', exist_ok=True)

# Preparar dados para salvar
data_to_save = {
    'text': df['text_processed'].values,
    'target': df['target'].values,
    'target_names': newsgroups.target_names
}

# Salvar em pickle
output_path = '../data/processed/20news_preprocessed.pkl'
with open(output_path, 'wb') as f:
    pickle.dump(data_to_save, f)

print(f"Dataset salvo em: {output_path}")
print(f"Total de documentos salvos: {len(data_to_save['text'])}")
print("Distribuição final das classes:\n")

# Contar ocorrências por classe
class_counts = pd.Series(data_to_save['target']).value_counts().sort_index()

# Criar DataFrame com nome da classe e contagem
class_distribution = pd.DataFrame({
    'Classe (index)': class_counts.index,
    'Nome da Classe': [data_to_save['target_names'][i] for i in class_counts.index],
    'Quantidade': class_counts.values
})

# Exibir
print(class_distribution.to_string(index=False))


Dataset salvo em: ../data/processed/20news_preprocessed.pkl
Total de documentos salvos: 5611
Distribuição final das classes:

 Classe (index)        Nome da Classe  Quantidade
              0             rec.autos         931
              1    rec.sport.baseball         951
              2      rec.sport.hockey         973
              3             sci.space         954
              4    talk.politics.guns         885
              5 talk.politics.mideast         917
